# Use Case Description

YARA is a well-known tool to identify and classify malware samples by describing patterns in textual or binary. <br>
But it's an onerous effort to list up bunch of patterns. <br>
Let's use our models to create YARA patterns and generate test strings.

## Model used for this use case
Instruct Model is ideal for generating snippets based on user instructions. In this example, we used the Instruct Model via SageMaker endpoint.

**Note**: Update the configuration variables below to match your deployment.

## Configuration
Update these variables to match your SageMaker deployment:

In [ ]:
# Update these variables to match your deployment
endpoint_name = 'foundation-sec-8b-endpoint'
aws_region = 'us-east-1'

print(f"Configuration:")
print(f"Endpoint: {endpoint_name}")
print(f"Region: {aws_region}")

## SetUp

The setup uses SageMaker endpoint instead of loading the model locally.

In [ ]:
import boto3
import json
import re
from IPython.display import display, Markdown

# Initialize SageMaker runtime client
sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=aws_region)

print(f"Connected to SageMaker endpoint: {endpoint_name}")

In [ ]:
# Generation arguments for consistent rule generation
generation_args = {
    "max_new_tokens": 1024,
    "temperature": None,  # None means deterministic (temperature=0)
    "repetition_penalty": 1.2,
    "do_sample": False,   # Deterministic sampling for consistent output
    "use_cache": True
}

print("Generation configuration:")
for key, value in generation_args.items():
    print(f"  {key}: {value}")

In [ ]:
def inference(prompt, system_prompt):
    """Inference function using SageMaker endpoint"""
    
    # Format the conversation for the model
    formatted_prompt = f"System: {system_prompt}\n\nUser: {prompt}\n\nAssistant: "
    
    # Prepare payload for SageMaker endpoint
    payload = {
        "inputs": formatted_prompt,
        "parameters": generation_args
    }
    
    try:
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Body=json.dumps(payload)
        )
        
        result = json.loads(response['Body'].read().decode())
        
        # Handle different TGI response formats
        if isinstance(result, list) and len(result) > 0:
            generated_text = result[0].get('generated_text', '')
        elif isinstance(result, dict):
            generated_text = result.get('generated_text', str(result))
        else:
            generated_text = str(result)
        
        # Clean up the response (remove the original prompt if it's included)
        if generated_text.startswith(formatted_prompt):
            response_text = generated_text[len(formatted_prompt):].strip()
        else:
            response_text = generated_text.strip()
            
        # Remove any trailing special tokens
        response_text = re.sub(r'<\|.*?\|>$', '', response_text).strip()
        
        return response_text
        
    except Exception as e:
        print(f"Error invoking endpoint: {str(e)}")
        return f"Error: {str(e)}"

# Test the inference function
test_response = inference("Hello, can you help with YARA rule generation?", "You are a security expert.")
print("Test Response:")
print(test_response[:200] + "..." if len(test_response) > 200 else test_response)
print("\nSageMaker inference function ready!")

## Generate YARA rules

Let's first create a YARA rule pattern to detect known malicious IPs or file hashes.

In [ ]:
SYSTEM_PROMPT = "You are a security expert and your task is write a draft YARA pattern to detect a simple malware sample."

user_prompt = '''Let's say 192.168.1.100 is a malicious IP and d41d8cd98f00b204e9800998ecf8427e is a file hash.
The YARA flags if either string being exactly matched in a given log file. No other conditions.'''

print("Generating YARA rule using SageMaker endpoint...")
yara_pattern = inference(user_prompt, SYSTEM_PROMPT)
print("YARA rule generated successfully!\n")
display(Markdown(yara_pattern))

## Make test logs and check by real YARA

First, ensure that the YARA pattern you just created conforms to the correct syntax.

In [ ]:
# Extract YARA rule from the response
match = re.search(r"```yara(.*?)```", yara_pattern, re.DOTALL)
if match:
    pattern = match.group(1).strip()
    print("Extracted YARA rule:")
    print(pattern)
else:
    # Fallback: try to extract without yara tags
    match = re.search(r"```(.*?)```", yara_pattern, re.DOTALL)
    if match:
        pattern = match.group(1).strip()
        print("Extracted rule (without yara tags):")
        print(pattern)
    else:
        print("Could not extract YARA rule from response. Please check the generated content.")
        pattern = None

In [ ]:
# Validate YARA syntax (requires yara-python package)
try:
    import yara
    if pattern:
        # If there's an error it shows that the pattern doesn't conform to the correct syntax.
        rules = yara.compile(source=pattern)
        print("✅ YARA rule syntax is valid!")
    else:
        print("❌ No pattern to validate")
except ImportError:
    print("⚠️  yara-python package not installed. Install with: pip install yara-python")
    print("Skipping syntax validation...")
    rules = None
except Exception as e:
    print(f"❌ YARA rule syntax error: {e}")
    rules = None

Let's also create some test strings to verify that the YARA patterns function as expected.

In [ ]:
if pattern:
    test_prompt = f'''Given the following YARA pattern, write two simple log files: first one is flagged by the YARA, while the second one is not flagged.
Since you are writing logs for unit tests, don't include any YARA related descriptions.
Each log file should be separated by ---.

{pattern}
'''
    
    NEW_SYSTEM_PROMPT = "You are a cybersecurity test engineer."
    
    print("Generating test logs...")
    test_response = inference(test_prompt, system_prompt=NEW_SYSTEM_PROMPT)
    print("Test logs generated!\n")
    print(test_response)
else:
    print("Cannot generate test logs without a valid YARA pattern.")
    test_response = None

In [ ]:
# Split the test logs
if test_response and "---" in test_response:
    log_to_be_flagged = test_response.split("---")[0].strip()
    log_not_to_be_flagged = test_response.split("---")[1].strip()
    
    print("Log that should be flagged:")
    print(log_to_be_flagged)
    print("\n" + "-"*50)
    print("Log that should NOT be flagged:")
    print(log_not_to_be_flagged)
else:
    print("Could not parse test logs. Expected format with '---' separator.")
    log_to_be_flagged = None
    log_not_to_be_flagged = None

The YARA pattern correctly flags the first log file but does not flag the second one as expected.

In [ ]:
# Test the YARA rule against the test logs
if rules and log_to_be_flagged:
    print("Testing YARA rule against logs...")
    
    log_to_be_flagged_matches = rules.match(data=log_to_be_flagged)
    print(f"\nLog that should be flagged: {log_to_be_flagged_matches}")
    
    if log_not_to_be_flagged:
        log_not_to_be_flagged_matches = rules.match(data=log_not_to_be_flagged)
        print(f"Log that should NOT be flagged: {log_not_to_be_flagged_matches}")
    
    # Verify expected behavior
    if log_to_be_flagged_matches and (not log_not_to_be_flagged or not log_not_to_be_flagged_matches):
        print("\n✅ YARA rule working as expected!")
    else:
        print("\n⚠️  YARA rule behavior doesn't match expectations.")
else:
    print("Cannot test YARA rule - missing rules or test data.")

## Advanced YARA Rule Generation

Let's create more sophisticated YARA rules for different scenarios:

In [ ]:
# Generate advanced YARA rules
advanced_scenarios = [
    {
        "name": "Ransomware Detection",
        "prompt": "Create a YARA rule to detect potential ransomware based on file extensions (.encrypted, .locked) and ransom note keywords (bitcoin, decrypt, payment)."
    },
    {
        "name": "Suspicious PowerShell",
        "prompt": "Create a YARA rule to detect suspicious PowerShell commands including base64 encoding, download cradles, and execution bypass techniques."
    },
    {
        "name": "C2 Communication",
        "prompt": "Create a YARA rule to detect command and control communication patterns including specific user agents, URL patterns, and POST data formats."
    }
]

advanced_rules = {}

for scenario in advanced_scenarios:
    print(f"\n=== Generating {scenario['name']} Rule ===")
    
    advanced_rule = inference(scenario['prompt'], SYSTEM_PROMPT)
    advanced_rules[scenario['name']] = advanced_rule
    
    print(f"{scenario['name']} rule generated.")
    display(Markdown(f"### {scenario['name']}\n{advanced_rule}"))
    print("-" * 60)

## Bulk YARA Rule Generation

Generate multiple YARA rules efficiently:

In [ ]:
def generate_yara_ruleset(ioc_list, rule_name_prefix="DetectionRule"):
    """Generate a comprehensive YARA ruleset from a list of IOCs"""
    
    ioc_strings = "\n".join([f"- {ioc}" for ioc in ioc_list])
    
    bulk_prompt = f"""Create a comprehensive YARA rule that detects any of the following indicators of compromise:
    
{ioc_strings}

The rule should:
1. Have a descriptive name starting with '{rule_name_prefix}'
2. Include metadata with description, author, and date
3. Use appropriate string definitions for each IOC
4. Have a condition that triggers on any match
5. Be syntactically correct for YARA
"""
    
    return inference(bulk_prompt, SYSTEM_PROMPT)

# Example IOC list
sample_iocs = [
    "malicious-domain.com",
    "192.168.1.100",
    "c2-server.evil.net",
    "5d41402abc4b2a76b9719d911017c592",  # MD5 hash
    "suspicious-file.exe",
    "backdoor.dll"
]

print("=== BULK YARA RULE GENERATION ===")
bulk_ruleset = generate_yara_ruleset(sample_iocs, "ThreatHunting")
display(Markdown(bulk_ruleset))

# Validate the bulk ruleset
try:
    import yara
    bulk_match = re.search(r"```yara(.*?)```", bulk_ruleset, re.DOTALL)
    if not bulk_match:
        bulk_match = re.search(r"```(.*?)```", bulk_ruleset, re.DOTALL)
    
    if bulk_match:
        bulk_pattern = bulk_match.group(1).strip()
        bulk_rules = yara.compile(source=bulk_pattern)
        print("\n✅ Bulk YARA ruleset syntax is valid!")
    else:
        print("\n⚠️  Could not extract YARA rule from bulk response")
except ImportError:
    print("\n⚠️  yara-python package not available for validation")
except Exception as e:
    print(f"\n❌ Bulk YARA ruleset syntax error: {e}")

## Custom YARA Rule Generation

Generate YARA rules for your specific use case:

In [ ]:
# Customize this section for your specific needs
custom_requirements = """
# Describe your YARA rule requirements here
# Example:
# I need a YARA rule to detect:
# - Specific malware family signatures
# - Network traffic patterns
# - File system artifacts
# - Registry modifications
"""

custom_iocs = [
    # Add your IOCs here
    # "your-malicious-domain.com",
    # "suspicious-hash-here",
    # "malicious-ip-address"
]

if custom_requirements.strip() and not custom_requirements.startswith("# Describe your"):
    print("=== CUSTOM YARA RULE GENERATION ===")
    
    custom_rule = inference(custom_requirements, SYSTEM_PROMPT)
    display(Markdown(custom_rule))
    
    # Validate custom rule
    try:
        import yara
        custom_match = re.search(r"```yara(.*?)```", custom_rule, re.DOTALL)
        if not custom_match:
            custom_match = re.search(r"```(.*?)```", custom_rule, re.DOTALL)
        
        if custom_match:
            custom_pattern = custom_match.group(1).strip()
            custom_rules_obj = yara.compile(source=custom_pattern)
            print("\n✅ Custom YARA rule syntax is valid!")
        else:
            print("\n⚠️  Could not extract YARA rule from custom response")
    except ImportError:
        print("\n⚠️  yara-python package not available for validation")
    except Exception as e:
        print(f"\n❌ Custom YARA rule syntax error: {e}")
        
elif custom_iocs:
    print("=== GENERATING RULES FROM IOC LIST ===")
    
    ioc_rule = generate_yara_ruleset(custom_iocs, "CustomDetection")
    display(Markdown(ioc_rule))
    
else:
    print("💡 Add your requirements or IOCs in the cells above to generate custom YARA rules!")

## YARA Rule Optimization and Analysis

Analyze and optimize existing YARA rules:

In [ ]:
def optimize_yara_rule(existing_rule):
    """Optimize an existing YARA rule for better performance and accuracy"""
    
    optimization_prompt = f"""Analyze and optimize the following YARA rule:
    
1. Identify potential performance issues
2. Suggest improvements for accuracy
3. Recommend additional conditions or metadata
4. Provide an optimized version of the rule
5. Explain the changes made

EXISTING RULE:
{existing_rule}
"""
    
    optimization_system = "You are a YARA rule optimization expert with deep knowledge of malware analysis and rule performance."
    
    return inference(optimization_prompt, optimization_system)

# Example: optimize the original rule we generated
if pattern:
    print("=== YARA RULE OPTIMIZATION ===")
    optimization_result = optimize_yara_rule(pattern)
    display(Markdown(optimization_result))
else:
    print("No rule available for optimization. Run the basic generation cells first.")

## Best Practices and Integration

### SageMaker Advantages for YARA Rule Generation

- **Scalability**: Generate large numbers of rules without local resource constraints
- **Cost Efficiency**: Pay only for generation time, no idle infrastructure costs
- **Consistency**: Deterministic rule generation for repeatable results
- **Integration**: Easy to integrate into automated threat intelligence workflows
- **Collaboration**: Share rule generation capabilities across security teams

### Integration Opportunities

This notebook can be integrated with:

- **Threat Intelligence Platforms**: Automated rule generation from IOC feeds
- **SIEM Systems**: Dynamic rule creation for new threats
- **Malware Analysis Platforms**: Automated signature generation
- **Security Orchestration**: Rule generation as part of incident response
- **Threat Hunting**: Custom rule creation for specific campaigns

### Quality Assurance for Generated Rules

1. **Syntax Validation**: Always validate with yara-python before deployment
2. **Performance Testing**: Test rules against large datasets for performance
3. **False Positive Analysis**: Validate against known good files
4. **Coverage Testing**: Ensure rules detect intended threats
5. **Regular Updates**: Refresh rules based on new threat intelligence

### Deployment Workflow

1. **Generate Rules**: Use this notebook for initial rule creation
2. **Validate Syntax**: Automated syntax checking
3. **Test Performance**: Benchmark against sample datasets
4. **Validate Accuracy**: Test against known samples
5. **Deploy to Production**: Gradual rollout with monitoring
6. **Monitor and Tune**: Continuous improvement based on results
